In [2]:
import pandas as pd

In [52]:
raw_data = pd.read_csv('./scraped_data_test.csv')
raw_data = raw_data.drop(["Unnamed: 0"], axis=1)
raw_data

,doc_id,dob,gender,race,facility,sentence_start,sentence_desc,conviction_type,citation_code,sentence_release,sentence_length_years,sentence_length_months,sentence_length_days
0,100000,05/1976,Male,Black,Returned to court authority on release,2011-07-18,FAILURE TO REGISTER AS A SEX OFFENDER,FD,11-8-8-17,2015-08-25,0,5,30
1,100000,05/1976,Male,Black,Returned to court authority on release,1999-12-22,RAPE,FB,35-42-4-1,2020-03-10,20,0,0
2,100000,05/1976,Male,Black,Returned to court authority on release,2016-03-21,FAILURE TO REGISTER AS A SEX OFFENDER,FD,11-8-8-17,2018-04-16,0,2,29
3,100000,05/1976,Male,Black,Returned to court authority on release,2011-07-18,FAILURE TO REGISTER AS A SEX OFFENDER,FD,11-8-8-17,2019-09-30,0,6,16
4,100000,05/1976,Male,Black,Returned to court authority on release,2020-08-24,FAILURE TO REGISTER AS A SEX OFFENDER,F5,11-8-8-17,2021-10-06,2,0,0


In [56]:
# Collapse concurrent sentences into one
def collapse_sentences(sentences):
    sentences = sentences.sort_values(by='sentence_start').reset_index()

    # Contains collapsed/merged sentences
    collapsed = sentences.copy()
    # Remove all but the earliest sentence (this will be the first one used for merging)
    collapsed = collapsed.drop(range(1,len(collapsed.index)))
        
    for i, row in sentences.iterrows():
        # Last row of collapsed df, representing most recent (merged) sentence
        last = len(collapsed.index) - 1
        
        # Sentences overlap if the release date of the first one is on or after the start date of the second one
        overlap = row['sentence_start'] <= collapsed.loc[last, 'sentence_release']   
         
        # If the end of this sentence overlaps with the last one, the two should be merged
        if overlap:
            collapsed.loc[last, 'sentence_release'] = row['sentence_release']
            collapsed.loc[last, 'sentence_desc'] += ',' + row['sentence_desc']
            collapsed.loc[last, 'conviction_type'] += ',' + row['conviction_type']
            collapsed.loc[last, 'citation_code'] += ',' + row['citation_code']
            collapsed.loc[last, 'sentence_length_years'] += row['sentence_length_years']
            collapsed.loc[last, 'sentence_length_months'] += row['sentence_length_months']
            collapsed.loc[last, 'sentence_length_days'] += row['sentence_length_days']
        
        # If there's no overlap, add this sentence separately
        else:
            collapsed.loc[last + 1] = row
    
    # An extra index column gets created for some reason
    collapsed = collapsed.drop(['index'], axis=1)
    
    return collapsed

collapsed_data = raw_data.groupby(['doc_id']).apply(collapse_sentences).reset_index(drop=True)
collapsed_data

,doc_id,dob,gender,race,facility,sentence_start,sentence_desc,conviction_type,citation_code,sentence_release,sentence_length_years,sentence_length_months,sentence_length_days
0,100000,05/1976,Male,Black,Returned to court authority on release,1999-12-22,"RAPE,RAPE,FAILURE TO REGISTER AS A SEX OFFENDE...","FB,FB,FD,FD,FD","35-42-4-1,35-42-4-1,11-8-8-17,11-8-8-17,11-8-8-17",2018-04-16,40,13,75
1,100000,05/1976,Male,Black,Returned to court authority on release,2020-08-24,FAILURE TO REGISTER AS A SEX OFFENDER,F5,11-8-8-17,2021-10-06,2,0,0
